In [68]:
def convert_btec_grade(grade):
    """
    Converts a BTEC grade string into a numeric value for comparison.
    This function tokenizes the input string and assigns each token a weight:
        D* -> 4
        D  -> 3
        M  -> 2
        P  -> 1
    The final score is the sum of these weights.

    For example:
      - "D*D*D*" => 4 + 4 + 4 = 12
      - "DDD"    => 3 + 3 + 3 = 9
      - "D*M P"  => 4 + 2 + 1 = 7  (if tokens are correctly extracted)
    """
    if not grade or not isinstance(grade, str):
        return None

    grade = grade.strip().upper()
    token_weights = {
        "D*": 4,
        "D": 3,
        "M": 2,
        "P": 1,
    }

    # Use regex to extract tokens; note that 'D*' should be matched before 'D'
    pattern = r'(D\*|D|M|P)'
    tokens = re.findall(pattern, grade)
    if not tokens:
        return None

    total_weight = sum(token_weights.get(token, 0) for token in tokens)
    return total_weight


In [69]:
def extract_gcse_grades(text):
    """
    Extracts numeric GCSE grades from a text description.
    For example, from "grade 5, grade 6, grade 7" it will extract [5, 6, 7].
    If no valid grade is found, returns an empty list.
    """
    if not text or not isinstance(text, str):
        return []
    # This regex looks for a digit (or digits) that come after the word "grade" (optional spaces).
    matches = re.findall(r'grade\s*(\d+)', text, flags=re.IGNORECASE)
    return [int(match) for match in matches]

In [70]:
def convert_bachelors_degree_from_gpa(gpa):
    """
    Converts a GPA value into a bachelor's degree classification string.

    Parameters:
        gpa (float or str): The GPA value. It will be converted to float.

    Returns:
        str: A classification such as "first class", "upper second class", etc.
             If the GPA does not meet any threshold, returns None.
    """
    try:
        gpa = float(gpa)
    except (TypeError, ValueError):
        return None

    if gpa >= 3.7:
        return "first class"
    elif gpa >= 3.3:
        return "upper second class"
    elif gpa >= 3.0:
        return "lower second class"
    elif gpa >= 2.5:
        return "third class"
    else:
        return None

In [29]:
import pandas as pd
import re


def extract_numeric(value):
    """Extracts the first numeric value from a string."""
    if pd.isna(value):
        return None
    match = re.search(r'\d+', str(value))
    return int(match.group()) if match else None


def convert_a_level_to_numeric(grade):
    """
    Converts an A-Level grade string (e.g., "A*A*A*", "A*A*A", "AAB") into a numeric value.
    This function breaks down the grade into individual components, assigns a numeric value to each,
    and sums them up.
    """
    if not grade or not isinstance(grade, str):
        return None

    # Define the numeric values for each individual grade component.
    grade_values = {
        "A*": 56,
        "A": 48,
        "B": 40,
        "C": 32,
        "D": 24,
        "E": 16
    }

    # Use regex to find all grade components.
    # The pattern looks for "A*" first (since it has a special character), then A, B, etc.
    pattern = r'(A\*|A|B|C|D|E)'
    matches = re.findall(pattern, grade.upper())

    if not matches:
        return None

    # Sum the numeric values for each grade component.
    numeric_score = sum(grade_values.get(match, 0) for match in matches)
    return numeric_score


def initial_filtering(file_path, filters):
    """
    Performs an initial filtering of the courses based on key parameters.
    Supports:
    - Finding universities based on a specific degree program.
    - Standard filtering using university name, field type, location, and degree program type.
    - Handles cases where a country name (e.g., "UK") is provided instead of a specific city.
    - Additional filtering for IELTS overall score and IELTS individual band scores.
    """

    # Load the Excel file
    df = pd.read_excel(file_path, sheet_name="Sheet1")

    # Normalize dataset column values (convert to lowercase for case-insensitive matching)
    column_list = ["university_name", "field_name", "location", "degree_program", "course_or_degree_name", "country",
                   "ielts", "ielts_individual_component", "ucas_tariff"]
    for col in column_list:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.lower()

    print("Initial course count:", df.shape[0])

    # Alias check: If "course name" is provided, map it to "course_or_degree_name"
    if "course name" in filters:
        filters["course_or_degree_name"] = filters.pop("course name")

    # Check if the user query is only asking for universities offering a specific course
    if filters.get("course_or_degree_name") and not any(
            key in filters for key in ["university name", "field type", "location", "degree program type"]
    ):
        course_filter = filters["course_or_degree_name"].strip().lower()
        df_filtered = df[df["course_or_degree_name"].notna() & df["course_or_degree_name"].str.contains(
            course_filter, case=False, na=False, regex=False)]
        print("After filtering by course name:", df_filtered.shape[0])

        # Return only unique universities that offer this course
        if not df_filtered.empty:
            return df_filtered[["university_name"]].drop_duplicates()

        return "No universities found for this course."

    # Apply standard filtering (when multiple filters are used)
    if filters.get("university_name"):
        df = df[df["university_name"] == filters["university_name"].strip().lower()]
        print("After university name filter:", df.shape[0])

    if filters.get("field_type"):
        df = df[df["field_name"] == filters["field_type"].strip().lower()]
        print("After field type filter:", df.shape[0])

    if filters.get("location"):
        location_filter = filters["location"].strip().lower()

        # Skip location filtering if the user provided "UK"
        if location_filter != "uk":
            df = df[df["location"] == location_filter]
            print("After location filter:", df.shape[0])

    if filters.get("degree program type"):
        df = df[df["degree_program"] == filters["degree program type"].strip().lower()]
        print("After degree program type filter:", df.shape[0])

    # Apply IELTS filtering
    if "ielts" in filters:
        user_ielts = float(filters["ielts"])
        df["ielts"] = pd.to_numeric(df["ielts"], errors='coerce')
        df = df[(df["ielts"].isna()) | (df["ielts"] <= user_ielts)]
        print("After IELTS filter:", df.shape[0])

    # Apply IELTS Individual Component filtering
    if "ielts_individual_component" in filters:
        user_ielts_component = float(filters["ielts_individual_component"])
        df["ielts_individual_component"] = pd.to_numeric(df["ielts_individual_component"], errors='coerce')
        df = df[(df["ielts_individual_component"].isna()) | (df["ielts_individual_component"] <= user_ielts_component)]
        print("After IELTS Individual Component filter:", df.shape[0])

    # Apply UCAS Tariff filtering
    if "ucas_tariff" in filters:
        user_ucas = int(filters["ucas_tariff"])
        df["ucas_tariff_numeric"] = df["ucas_tariff"].apply(extract_numeric)
        df = df[(df["ucas_tariff_numeric"].isna()) | (df["ucas_tariff_numeric"] <= user_ucas)]
        print("After UCAS Tariff filter:", df.shape[0])

    if "btec_diploma" in filters:
        user_btec = convert_btec_grade(filters["btec_diploma"].upper())
        df["btec_diploma_numeric"] = df["btec_diploma"].apply(lambda x: convert_btec_grade(str(x).upper()))
        df = df[(df["btec_diploma_numeric"].isna()) | (df["btec_diploma_numeric"] <= user_btec)]
        print("After BTEC Diploma filter:", df.shape[0])

    # Apply BTEC Extended Diploma Filtering
    if "btec_extended_diploma" in filters:
        user_extended_btec = convert_btec_grade(filters["btec_extended_diploma"].upper())
        df["btec_extended_diploma_numeric"] = df["btec_extended_diploma"].apply(
            lambda x: convert_btec_grade(str(x).upper()))
        df = df[
            (df["btec_extended_diploma_numeric"].isna()) | (df["btec_extended_diploma_numeric"] <= user_extended_btec)]
        print("After BTEC Extended Diploma filter:", df.shape[0])

    # Apply GCSE Overall Filtering
    if "gcse_overall" in filters:
        user_gcse_grades = extract_gcse_grades(filters["gcse_overall"])
        df["gcse_grades"] = df["gcse_overall"].apply(extract_gcse_grades)
        df = df[df["gcse_grades"].apply(lambda grades: all(g >= 4 for g in grades))]
        print("After GCSE Overall filter:", df.shape[0])

    # Apply Mandatory Subject of GCSE Filtering
    if "mandatory_subject_of_gcse" in filters:
        required_subjects = set(filters["mandatory_subject_of_gcse"].lower().split(", "))
        df = df[df["mandatory_subject_of_gcse"].apply(
            lambda x: required_subjects.issubset(set(str(x).lower().split(", "))))]
        print("After Mandatory GCSE Subject filter:", df.shape[0])

    # Apply A-Levels filtering using the improved conversion function
    if "a_levels_overall" in filters:
        # Convert the user's A-Level grade to its numeric value
        user_grade = filters["a_levels_overall"]
        user_a_level = convert_a_level_to_numeric(user_grade.upper())

        if user_a_level is not None:
            # Ensure that each A-Level value in the DataFrame is treated as a string before conversion.
            df["a_levels_numeric"] = df["a_levels_overall"].astype(str).apply(
                lambda x: convert_a_level_to_numeric(x.upper()))
            df = df[(df["a_levels_numeric"].isna()) | (df["a_levels_numeric"] <= user_a_level)]
            print("After A-Levels filter:", df.shape[0])
        else:
            print("Invalid user A-Level grade provided:", user_grade)

        if "btec_diploma" in filters:
            user_btec = convert_btec_grade(filters["btec_diploma"].upper())
        df["btec_diploma_numeric"] = df["btec_diploma"].apply(lambda x: convert_btec_grade(str(x).upper()))
        df = df[(df["btec_diploma_numeric"].isna()) | (df["btec_diploma_numeric"] <= user_btec)]
        print("After BTEC Diploma filter:", df.shape[0])

    if "btec_extended_diploma" in filters:
        user_extended_btec = convert_btec_grade(filters["btec_extended_diploma"].upper())
        df["btec_extended_diploma_numeric"] = df["btec_extended_diploma"].apply(
            lambda x: convert_btec_grade(str(x).upper()))
        df = df[
            (df["btec_extended_diploma_numeric"].isna()) | (df["btec_extended_diploma_numeric"] <= user_extended_btec)]
        print("After BTEC Extended Diploma filter:", df.shape[0])

    if "gcse_overall" in filters:
        user_gcse_grades = extract_gcse_grades(filters["gcse_overall"])
        # For each course, extract the list of GCSE grades
        df["gcse_grades"] = df["gcse_overall"].apply(extract_gcse_grades)
        # Example logic: only keep courses where every GCSE grade is at least 4
        df = df[df["gcse_grades"].apply(lambda grades: all(g >= 4 for g in grades))]
        print("After GCSE Overall filter:", df.shape[0])

    if "mandatory_subject_of_gcse" in filters:
        required_subjects = set(filters["mandatory_subject_of_gcse"].lower().split(", "))
        df = df[df["mandatory_subject_of_gcse"].apply(
            lambda x: required_subjects.issubset(set(str(x).lower().split(", ")))
        )]
        print("After Mandatory GCSE Subject filter:", df.shape[0])

    # Return results
    if not df.empty and "course_or_degree_name" in df.columns:
        return df[["university_name", "course_or_degree_name"]]
    else:
        return "No matching results found."


In [35]:
file_path = "../data/test_data.xlsx"

In [66]:
filters = {
    "a_levels_overall": "C"
}

In [67]:
initial_filtering(file_path, filters)

Initial course count: 50
After A-Levels filter: 0


'No matching results found.'

In [59]:
convert_a_level_to_numeric("CCE")

80